#### Суть стратегии:
При положительном спреде одновременно:
* Покупаем актив на споте за 10,000 USDT
* Открываем шорт на перпетуале на 10,000 USDT (с плечом x10, используя 1,000 USDT залога)
* Прибыль = разница в ценах × количество монет

## 1. Инициализация
* Загружаем список 200 криптовалютных пар
* Устанавливаем параметры: 10,000 USDT на спот + 1,000 USDT на перпетуалы (плечо x10)
* Настраиваем интервалы обновления и пороги для подсветки

## 2. Получение данных с биржи
* Спот рынок: запрос к API https://api.bybit.com/v5/market/tickers?category=spot
* Перпетуальные фьючерсы: запрос к API https://api.bybit.com/v5/market/tickers?category=linear
* Извлекаем цены последних сделок для каждого актива

## 3. Фильтрация возможностей
* Проверяем условие: Цена_спота < Цена_перпетуала
* Отбрасываем все пары где спред отрицательный или нулевой
* Остаются только возможности для стратегии "покупка спота + шорт перпетуала"

## 4. Расчет ключевых метрик
Спред в процентах:
* Спред% = (Цена_перпетуала - Цена_спота) / Цена_спота × 100
Количество монет для торговли:
* Количество_спот = 10,000 USDT / Цена_спота
* Количество_перп = 10,000 USDT / Цена_перпетуала
* Количество_торговли = min(Количество_спот, Количество_перп)
Потенциальная прибыль:
* Прибыль = (Цена_перпетуала - Цена_спота) × Количество_торговли

## 5. Сортировка и отображение
* Сортирует все возможности по размеру спреда (от большего к меньшему)
* Показывает топ-50 лучших возможностей

## 6. Цикл обновления
* Повторяем весь процесс каждые 3 секунды

In [ ]:
import requests
import time
import json
from datetime import datetime
from colorama import init, Fore, Back, Style
import os

# Инициализация colorama для цветного вывода
init(autoreset=True)

# ==================== НАСТРОЙКИ ====================
# Торговые пары для мониторинга (200 активов - доступные на споте и perpetual)
SYMBOLS = [
    # Топ криптовалюты
    'BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'XRPUSDT', 'ADAUSDT', 'SOLUSDT', 'DOGEUSDT', 'DOTUSDT',
    'MATICUSDT', 'AVAXUSDT', 'LTCUSDT', 'LINKUSDT', 'BCHUSDT', 'XLMUSDT', 'ATOMUSDT', 'UNIUSDT',
    'ETCUSDT', 'TRXUSDT', 'FILUSDT', 'VETUSDT', 'ICPUSDT', 'NEOUSDT', 'EOSUSDT', 'XTZUSDT',
    'AAVEUSDT', 'ALGOUSDT', 'THETAUSDT', 'MKRUSDT', 'COMPUSDT', 'SNXUSDT', 'YFIUSDT', 'SUSHIUSDT',
    
    # DeFi токены
    'CRVUSDT', '1INCHUSDT', 'LRCUSDT', 'BATUSDT', 'MANAUSDT', 'SANDUSDT', 'ENJUSDT', 'CHZUSDT',
    'AXSUSDT', 'GALAUSDT', 'FLOWUSDT', 'NEARUSDT', 'FTMUSDT', 'KSMUSDT', 'RUNEUSDT', 'IMXUSDT',
    'OMGUSDT', 'SCUSDT', 'IOTXUSDT', 'DASHUSDT', 'ONEUSDT', 'STXUSDT', 'ZECUSDT', 'ANKRUSDT',
    'GRTUSDT', 'DYDXUSDT', 'RSRUSDT', 'SRMUSDT', 'IOSTUSDT', 'CELRUSDT', 'STORJUSDT', 'AUDIOUSDT',
    
    # Gaming и NFT
    'COTIUSDT', 'CHRUSDT', 'CVCUSDT', 'WOOUSDT', 'ALICEUSDT', 'ENSUSDT', 'C98USDT', 'YGGUSDT',
    'ILVUSDT', 'RNDRUSDT', 'KEEPUSDT', 'MASKUSDT', 'TLMUSDT', 'SLPUSDT', 'GTCUSDT', 'LITUSDT',
    'CTKUSDT', 'BICOUSDT', 'SXPUSDT', 'BTTUSDT', 'BSVUSDT', 'KLAYUSDT', 'QTUMUSDT', 'LPTUSDT',
    'SPELLUSDT', 'HBARUSDT', 'EGLDUSDT', 'ZENUSDT', 'REQUSDT', 'XEMUSDT', 'DENTUSDT', 'HOTUSDT',
    
    # Layer 1 и Layer 2
    'OPUSDT', 'ARBUSDT', 'APTUSDT', 'SUIUSDT', 'INJUSDT', 'SEIUSDT', 'TIAUSDT', 'BLURUSDT',
    'JUPUSDT', 'DYMUSDT', 'WLDUSDT', 'ORDIUSDT', 'FETUSDT', 'AGIXUSDT', 'OCEANUSDT', 'TAOUSDT',
    'LDOUSDT', 'STRKUSDT', 'ARUSDT', 'ALTUSDT', 'PYTHUSDT', 'RONINUSDT', 'PIXELUSDT', 'ACEUSDT',
    'NFPUSDT', 'AIUSDT', 'XAIUSDT', 'MANTAUSDT', 'ONDOUSDT', 'LSKUSDT', 'PORTALUSDT', 'PDAUSDT',
    
    # Мемкоины и трендовые
    'PEPEUSDT', 'SHIBUSDT', 'FLOKIUSDT', 'BONKUSDT', 'WIFUSDT', 'BOMEUSDT', 'NOTUSDT', 'DOGSUSDT',
    'POPCATUSDT', 'MEWUSDT', 'CATUSDT', 'ACTUSDT', 'PNUTUSDT', 'GOATUSDT', 'MOVEUSDT', 'MEUSDT',
    'TRUMPUSDT', 'CHILLGUYUSDT', 'XRPUSDT', 'HIPPOUSDT', 'GRASSUSDT', 'SCRUSDT', 'PENGUUSDT', 'HYPEUSDT',
    'AERODROMEUSDT', 'OWLSFMUSDT', 'VOXUSDT', 'RAYDIUMUSDT', 'SPIRITUSDT', 'LOOKUSDT', 'RADUSDT', 'GMXUSDT',
    
    # Стейблкоины и другие активы
    'LEVERUSDT', 'BADGERUSDT', 'FARMRUSDT', 'RAREUSDT', 'FIDAUSDT', 'OXTUSDT', 'ERNUSDT', 'KLNUSDT',
    'PERPUSDT', 'RAMPUSDT', 'SUPERUSDT', 'CFXUSDT', 'STGUSDT', 'LQTYUSDT', 'AMBUSDT', 'GASUSDT',
    'GLMRUSDT', 'STRAXUSDT', 'LOKAUSDT', 'API3USDT', 'BNTUSDT', 'WAXPUSDT', 'TRIBEUSDT', 'GNOUSDT',
    'POLYUSDT', 'POLYXUSDT', 'FXSUSDT', 'EURUSDT', 'MATIC2USDT', 'RIFUSDT', 'CITYUSDT', 'LEOUSDT',
    
    # Дополнительные популярные токены
    'HMSTRUSDT', 'EIGENUSDT', 'GRASSUSDT', 'SCRUSDT', 'LUMAUSDT', 'POLUSDT', 'NEIROUSDT', 'ACTUSDT',
    'PNUTUSDT', 'GOATUSDT', 'MOVEUSDT', 'MEUSDT', 'CATUSDT', 'PONKEUSDT', 'LUCE', 'CHEETAUSDT',
    'KAIAUSDT', 'ACXUSDT', 'ORCAUSDT', 'ELONUSDT', 'KEKUSDT', 'DEGUSDT', 'TRENDUSDT', 'CHEEMSUSDT',
    'BABYDOGEUSDT', 'KISHUUSDT', 'SAFEMOONUSDT', 'AKITAUSDT', 'HOKUSDT', 'BABYSHIBUSDT', 'MILADY',
    
    # Технологические и утилитарные токены
    'RENDERUSDT', 'ARKMUSDT', 'AIUSDT', 'PHBUSDT', 'ARKUSDT', 'CVXUSDT', 'LDOUSDT', 'RDNTUSDT',
    'COCOSUSDT', 'LOOKSUSDT', 'DAPPUSDT', 'FIDAUSDT', 'JASMYUSDT', 'HFTUSDT', 'HOOKUSDT', 'MAGICUSDT',
    'HIGHUSDT', 'AGTUSDT', 'PHAUSDT', 'GMTUSDT', 'KDAUSDT', 'APEUSDT', 'ROSEUSDT', 'DUSKUSDT',
    'REEFUSDT', 'WINUSDT', 'KEYUSDT', 'STMXUSDT', 'DREPUSDT', 'WRXUSDT', 'WTCUSDT', 'BCDUSDT'
]

# Финансовые параметры
SPOT_CAPITAL = 10000  # USDT на споте
PERPETUAL_CAPITAL = 1000  # USDT капитал для перпетуалов
LEVERAGE = 10  # Плечо для перпетуалов
PERPETUAL_POSITION_SIZE = PERPETUAL_CAPITAL * LEVERAGE  # 10000 USDT позиция

# Настройки мониторинга
UPDATE_INTERVAL = 3  # Секунды между обновлениями (увеличено из-за большого количества активов)
MIN_SPREAD_HIGHLIGHT = 0.3  # Минимальный спред в % для выделения (снижено для большей чувствительности)
PROFIT_HIGHLIGHT_THRESHOLD = 30  # Минимальная прибыль в USDT для выделения (снижено)
MAX_DISPLAY_ITEMS = 50  # Максимальное количество отображаемых активов (для удобства чтения)

# API endpoints
SPOT_API_URL = "https://api.bybit.com/v5/market/tickers"
PERPETUAL_API_URL = "https://api.bybit.com/v5/market/tickers"

# ==================== ОСНОВНОЙ КОД ====================

class BybitSpreadMonitor:
    def __init__(self):
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        })
    
    def get_spot_prices(self):
        """Получить цены спот рынка"""
        try:
            params = {
                'category': 'spot'
            }
            response = self.session.get(SPOT_API_URL, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()
            
            prices = {}
            if data.get('retCode') == 0:
                for ticker in data['result']['list']:
                    symbol = ticker['symbol']
                    if symbol in SYMBOLS:
                        prices[symbol] = {
                            'price': float(ticker['lastPrice']),
                            'bid': float(ticker['bid1Price']) if ticker['bid1Price'] else 0,
                            'ask': float(ticker['ask1Price']) if ticker['ask1Price'] else 0
                        }
            return prices
        except Exception as e:
            print(f"Ошибка получения спот цен: {e}")
            return {}
    
    def get_perpetual_prices(self):
        """Получить цены перпетуальных фьючерсов"""
        try:
            params = {
                'category': 'linear'
            }
            response = self.session.get(PERPETUAL_API_URL, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()
            
            prices = {}
            if data.get('retCode') == 0:
                for ticker in data['result']['list']:
                    symbol = ticker['symbol']
                    if symbol in SYMBOLS:
                        prices[symbol] = {
                            'price': float(ticker['lastPrice']),
                            'bid': float(ticker['bid1Price']) if ticker['bid1Price'] else 0,
                            'ask': float(ticker['ask1Price']) if ticker['ask1Price'] else 0
                        }
            return prices
        except Exception as e:
            print(f"Ошибка получения цен перпетуалов: {e}")
            return {}
    
    def calculate_spread_and_profit(self, spot_price, perp_price):
        """Рассчитать спред и потенциальную прибыль"""
        if spot_price <= 0 or perp_price <= 0:
            return None, None, None
        
        # Спред в %: (перпетуал - спот) / спот * 100
        spread_percent = ((perp_price - spot_price) / spot_price) * 100
        
        # Количество монет, которые можем купить на споте
        spot_quantity = SPOT_CAPITAL / spot_price
        
        # Количество монет в перпетуальной позиции
        perp_quantity = PERPETUAL_POSITION_SIZE / perp_price
        
        # Используем минимальное количество для симметричной позиции
        trade_quantity = min(spot_quantity, perp_quantity)
        
        # Потенциальная прибыль при закрытии позиций
        # Покупаем спот по текущей цене, шортим перп по текущей цене
        # Прибыль = (perp_entry_price - spot_entry_price) * quantity
        potential_profit = (perp_price - spot_price) * trade_quantity
        
        return spread_percent, potential_profit, trade_quantity
    
    def format_price(self, price):
        """Форматировать цену"""
        if price >= 1000:
            return f"{price:,.2f}"
        elif price >= 1:
            return f"{price:.4f}"
        else:
            return f"{price:.6f}"
    
    def display_data(self, spot_prices, perp_prices):
        """Отобразить данные с цветовой подсветкой"""
        # Очистить экран
        os.system('cls' if os.name == 'nt' else 'clear')
        
        print(f"{Fore.CYAN}{'='*100}")
        print(f"{Fore.CYAN}BYBIT SPREAD MONITOR - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"{Fore.CYAN}Капитал: Спот {SPOT_CAPITAL:,} USDT | Перпетуал {PERPETUAL_CAPITAL:,} USDT (плечо x{LEVERAGE})")
        print(f"{Fore.CYAN}МОНИТОРИНГ {len(SYMBOLS)} АКТИВОВ - ПОКАЗЫВАЕМ ТОЛЬКО СПОТ < PERPETUAL")
        print(f"{Fore.CYAN}{'='*100}")
        
        print(f"\n{Fore.WHITE}{Style.BRIGHT}{'#':<3} {'Символ':<12} {'Спот':<12} {'Перпетуал':<12} {'Спред %':<10} {'Прибыль USDT':<15} {'Количество':<12}")
        print(f"{Fore.WHITE}{'-'*90}")
        
        spreads_data = []
        
        for symbol in SYMBOLS:
            if symbol in spot_prices and symbol in perp_prices:
                spot_price = spot_prices[symbol]['price']
                perp_price = perp_prices[symbol]['price']
                
                # ФИЛЬТР: показываем только когда спот меньше перпетуала
                if spot_price < perp_price:
                    spread_percent, profit, quantity = self.calculate_spread_and_profit(spot_price, perp_price)
                    
                    if spread_percent is not None and spread_percent > 0:
                        spreads_data.append({
                            'symbol': symbol,
                            'spread': spread_percent,
                            'profit': profit,
                            'spot_price': spot_price,
                            'perp_price': perp_price,
                            'quantity': quantity
                        })
        
        # Сортировать по спреду (по убыванию) и ограничить количество отображаемых элементов
        spreads_data.sort(key=lambda x: x['spread'], reverse=True)
        display_data = spreads_data[:MAX_DISPLAY_ITEMS]  # Показываем только топ результаты
        
        # Отображение результатов с нумерацией
        for idx, data in enumerate(display_data, 1):
            symbol = data['symbol']
            spread = data['spread']
            profit = data['profit']
            spot_price = data['spot_price']
            perp_price = data['perp_price']
            quantity = data['quantity']
            
            # Цветовая подсветка для положительных спредов
            color = Fore.GREEN
            highlight = " 📈"
            
            if spread >= MIN_SPREAD_HIGHLIGHT:
                color = Fore.GREEN + Style.BRIGHT
                highlight += " 🔥"
            
            if profit >= PROFIT_HIGHLIGHT_THRESHOLD:
                color = Fore.YELLOW + Style.BRIGHT
                highlight += " 💰"
            
            # Дополнительная подсветка для топ-5
            if idx <= 5:
                color = Fore.MAGENTA + Style.BRIGHT
                if idx == 1:
                    highlight += f" 🥇"
                elif idx == 2:
                    highlight += f" 🥈"
                elif idx == 3:
                    highlight += f" 🥉"
                else:
                    highlight += f" 🏆"
            
            spread_str = f"{spread:.2f}%"
            profit_str = f"{profit:.2f}"
            
            print(f"{color}{idx:<3} {symbol:<12} {self.format_price(spot_price):<12} {self.format_price(perp_price):<12} "
                  f"{spread_str:<10} {profit_str:<15} {quantity:.4f}{highlight}")
        
        # Показать сводку по остальным активам если их больше чем отображается
        if len(spreads_data) > MAX_DISPLAY_ITEMS:
            remaining = len(spreads_data) - MAX_DISPLAY_ITEMS
            print(f"\n{Fore.CYAN}... и еще {remaining} возможностей (показаны топ-{MAX_DISPLAY_ITEMS})")
        
        # Расширенная статистика для 200 активов
        if spreads_data:
            best_item = max(spreads_data, key=lambda x: x['spread'])
            total_opportunities = len(spreads_data)
            avg_spread = sum(x['spread'] for x in spreads_data) / len(spreads_data)
            total_potential_profit = sum(x['profit'] for x in spreads_data)
            
            # Топ-5 возможности
            top_5 = spreads_data[:5]
            
            print(f"\n{Fore.CYAN}📊 СТАТИСТИКА ПО {len(SYMBOLS)} АКТИВАМ:")
            print(f"{Fore.GREEN}🥇 Лучшая возможность: {best_item['symbol']} "
                  f"(спред: {best_item['spread']:.2f}%, прибыль: {best_item['profit']:.2f} USDT)")
            
            if len(top_5) >= 2:
                print(f"{Fore.YELLOW}🥈 Вторая: {top_5[1]['symbol']} "
                      f"(спред: {top_5[1]['spread']:.2f}%, прибыль: {top_5[1]['profit']:.2f} USDT)")
            
            if len(top_5) >= 3:
                print(f"{Fore.BLUE}🥉 Третья: {top_5[2]['symbol']} "
                      f"(спред: {top_5[2]['spread']:.2f}%, прибыль: {top_5[2]['profit']:.2f} USDT)")
            
            print(f"{Fore.CYAN}📈 Найдено возможностей: {total_opportunities} из {len(SYMBOLS)} активов")
            print(f"{Fore.CYAN}📊 Средний спред: {avg_spread:.2f}%")
            print(f"{Fore.CYAN}💰 Общий потенциал прибыли: {total_potential_profit:.2f} USDT")
            
            # Процентное соотношение возможностей
            opportunity_percentage = (total_opportunities / len(SYMBOLS)) * 100
            print(f"{Fore.CYAN}🎯 Процент активных возможностей: {opportunity_percentage:.1f}%")
            
            # Категории по размеру спреда
            high_spread = len([x for x in spreads_data if x['spread'] >= 1.0])
            medium_spread = len([x for x in spreads_data if 0.5 <= x['spread'] < 1.0])
            low_spread = len([x for x in spreads_data if x['spread'] < 0.5])
            
            print(f"{Fore.CYAN}📋 Распределение по спредам:")
            print(f"{Fore.RED}   Высокий спред (>1%): {high_spread} активов")
            print(f"{Fore.YELLOW}   Средний спред (0.5-1%): {medium_spread} активов") 
            print(f"{Fore.GREEN}   Низкий спред (<0.5%): {low_spread} активов")
            
        else:
            print(f"\n{Fore.RED}❌ НЕТ ВОЗМОЖНОСТЕЙ")
            print(f"{Fore.YELLOW}Сейчас нет активов где спот < перпетуал из {len(SYMBOLS)} отслеживаемых")
        
        print(f"\n{Fore.YELLOW}💡 Стратегия: ПОКУПКА СПОТА + ШОРТ ПЕРПЕТУАЛА")
        print(f"{Fore.YELLOW}💡 Положительный спред означает потенциальную прибыль")
        print(f"\n{Fore.CYAN}Следующее обновление через {UPDATE_INTERVAL} сек...")
    
    def run(self):
        """Запустить мониторинг"""
        print(f"{Fore.CYAN}🚀 Запуск мониторинга спредов Bybit...")
        print(f"{Fore.CYAN}📊 Отслеживаемые активы: {len(SYMBOLS)} пар")
        print(f"{Fore.CYAN}💰 Капитал: {SPOT_CAPITAL:,} USDT спот + {PERPETUAL_CAPITAL:,} USDT перпетуал (x{LEVERAGE})")
        print(f"{Fore.CYAN}⏱️  Интервал обновления: {UPDATE_INTERVAL} сек")
        print(f"{Fore.CYAN}📋 Показываем топ-{MAX_DISPLAY_ITEMS} возможностей")
        print(f"{Fore.YELLOW}⚡ Нажмите Ctrl+C для остановки")
        
        try:
            while True:
                spot_prices = self.get_spot_prices()
                perp_prices = self.get_perpetual_prices()
                
                if spot_prices and perp_prices:
                    self.display_data(spot_prices, perp_prices)
                else:
                    print(f"{Fore.RED}❌ Ошибка получения данных. Повтор через {UPDATE_INTERVAL} сек...")
                
                time.sleep(UPDATE_INTERVAL)
                
        except KeyboardInterrupt:
            print(f"\n{Fore.YELLOW}⏹️  Мониторинг остановлен пользователем")
        except Exception as e:
            print(f"\n{Fore.RED}💥 Критическая ошибка: {e}")

if __name__ == "__main__":
    monitor = BybitSpreadMonitor()
    monitor.run()

🚀 Запуск мониторинга спредов Bybit...
📊 Отслеживаемые активы: 255 пар
💰 Капитал: 10,000 USDT спот + 1,000 USDT перпетуал (x10)
⏱️  Интервал обновления: 3 сек
📋 Показываем топ-50 возможностей
⚡ Нажмите Ctrl+C для остановки
BYBIT SPREAD MONITOR - 2025-07-03 12:57:24
Капитал: Спот 10,000 USDT | Перпетуал 1,000 USDT (плечо x10)
МОНИТОРИНГ 255 АКТИВОВ - ПОКАЗЫВАЕМ ТОЛЬКО СПОТ < PERPETUAL

#   Символ       Спот         Перпетуал    Спред %    Прибыль USDT    Количество  
------------------------------------------------------------------------------------------
1   XEMUSDT      0.001840     0.001849     0.49%      48.67           5408328.8264 📈 🔥 💰 🥇
2   PORTALUSDT   0.043520     0.043570     0.11%      11.48           229515.7218 📈 🥈
3   FLOWUSDT     0.335100     0.335400     0.09%      8.94            29815.1461 📈 🥉
4   ORDIUSDT     7.8500       7.8530       0.04%      3.82            1273.3987 📈 🏆
5   EIGENUSDT    1.1430       1.1432       0.02%      1.75            8747.3758 📈 🏆

📊 СТАТИС